In [2]:
import csv
from operator import itemgetter
import networkx as nx
from networkx.algorithms import community

# Read data

In [3]:
with open('www.forefdn.org - Nodes.csv', 'r') as nodecsv: # Open the file
    nodereader = csv.reader(nodecsv) # Read the csv
    # Retrieve the data (using Python list comprhension and list slicing to remove the header row, see footnote 3)
    nodes = [n for n in nodereader][1:]

node_names = [n[0] for n in nodes] # Get a list of only the node names

In [4]:
with open('www.forefdn.org - Edges (Internal).csv', 'r') as edgecsv: # Open the file
    edgereader = csv.reader(edgecsv) # Read the csv
    edges = [tuple(e) for e in edgereader][1:] # Retrieve the data

# Build Network

In [5]:
G = nx.Graph()
G.add_nodes_from(node_names)
G.add_edges_from(edges)

In [6]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 537
Number of edges: 5147
Average degree:  19.1695


In [7]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')

# Metrics

In [8]:
density = nx.density(G)
print("Network density:", density)

Network density: 0.035763917840962785


In [9]:
triadic_closure = nx.transitivity(G)
print("Triadic closure:", triadic_closure)

Triadic closure: 0.28375687008112066


In [10]:
betweenness_dict = nx.betweenness_centrality(G) 
eigenvector_dict = nx.eigenvector_centrality(G)

In [11]:
nx.set_node_attributes(G, betweenness_dict, 'betweenness')
nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')

In [12]:
sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)

print("Top 20 nodes by betweenness centrality:")
for b in sorted_betweenness[:20]:
    print(b)

Top 20 nodes by betweenness centrality:
('https://forefdn.org/our-grantees/', 0.317984102672306)
('https://forefdn.org/resources/', 0.053143548290689756)
('https://forefdn.org/about-us/', 0.05167517931577339)
('https://forefdn.org/news-updates/', 0.03480411522596907)
('https://forefdn.org/tapping-into-virtual-recovery-supports/', 0.033945256797021074)
('https://forefdn.org/grants-and-funding/', 0.032754579009696395)
('https://forefdn.org/grantee-spotlight-encouraging-physicians-to-treat-opioid-use-disorder-in-emergency-departments-across-the-u-s/', 0.030246594960747854)
('https://forefdn.org/presidents-update-dec-20-fores-year-of-firsts-in-the-midst-of-a-pandemic/', 0.028722318484881364)
('https://forefdn.org/grantee-spotlight-improving-access-to-recovery-supports-in-colleges-and-universities/', 0.02561908783909677)
('https://forefdn.org/our-grantees/page/4/', 0.0202063691290673)
('https://forefdn.org/our-grantees/page/5/', 0.0202063691290673)
('https://forefdn.org/our-grantees/page/2/

In [13]:
#First get the top 20 nodes by betweenness as a list
top_betweenness = sorted_betweenness[:20]

#Then find and print their degree
for tb in top_betweenness: # Loop through top_betweenness
    degree = degree_dict[tb[0]] # Use degree_dict to access a node's degree, see footnote 2
    print("Name:", tb[0], "| Betweenness Centrality:", tb[1], "| Degree:", degree)

Name: https://forefdn.org/our-grantees/ | Betweenness Centrality: 0.317984102672306 | Degree: 272
Name: https://forefdn.org/resources/ | Betweenness Centrality: 0.053143548290689756 | Degree: 165
Name: https://forefdn.org/about-us/ | Betweenness Centrality: 0.05167517931577339 | Degree: 161
Name: https://forefdn.org/news-updates/ | Betweenness Centrality: 0.03480411522596907 | Degree: 159
Name: https://forefdn.org/tapping-into-virtual-recovery-supports/ | Betweenness Centrality: 0.033945256797021074 | Degree: 46
Name: https://forefdn.org/grants-and-funding/ | Betweenness Centrality: 0.032754579009696395 | Degree: 156
Name: https://forefdn.org/grantee-spotlight-encouraging-physicians-to-treat-opioid-use-disorder-in-emergency-departments-across-the-u-s/ | Betweenness Centrality: 0.030246594960747854 | Degree: 44
Name: https://forefdn.org/presidents-update-dec-20-fores-year-of-firsts-in-the-midst-of-a-pandemic/ | Betweenness Centrality: 0.028722318484881364 | Degree: 54
Name: https://fore

# Community Detection

In [14]:
communities = community.greedy_modularity_communities(G)

In [15]:
modularity_dict = {} # Create a blank dictionary
for i,c in enumerate(communities): # Loop through the list of communities, keeping track of the number for the community
    for name in c: # Loop through each person in a community
        modularity_dict[name] = i # Create an entry in the dictionary for the person, where the value is which group they belong to.

# Now you can add modularity information like we did the other metrics
nx.set_node_attributes(G, modularity_dict, 'modularity')

In [16]:
# First get a list of just the nodes in that class
class0 = [n for n in G.nodes() if G.nodes[n]['modularity'] == 0]

# Then create a dictionary of the eigenvector centralities of those nodes
class0_eigenvector = {n:G.nodes[n]['eigenvector'] for n in class0}

# Then sort that dictionary and print the first 5 results
class0_sorted_by_eigenvector = sorted(class0_eigenvector.items(), key=itemgetter(1), reverse=True)

print("Modularity Class 0 Sorted by Eigenvector Centrality:")
for node in class0_sorted_by_eigenvector[:5]:
    print("Name:", node[0], "| Eigenvector Centrality:", node[1])

Modularity Class 0 Sorted by Eigenvector Centrality:
Name: https://forefdn.org/news-updates/ | Eigenvector Centrality: 0.16320652670136257
Name: https://forefdn.org/about-us/ | Eigenvector Centrality: 0.16197041395824832
Name: https://forefdn.org/grants-and-funding/ | Eigenvector Centrality: 0.16180245675435415
Name: https://forefdn.org/ | Eigenvector Centrality: 0.16157382999812975
Name: https://forefdn.org/careers/ | Eigenvector Centrality: 0.16157382999812975


In [17]:
for i,c in enumerate(communities): # Loop through the list of communities
    if len(c) > 2: # Filter out modularity classes with 2 or fewer nodes
        print('Class '+str(i)+':', list(c)) # Print out the classes and their members

Class 0: ['https://forefdn.org/wp-content/uploads/2021/06/Stephen-Taylor-e1622734451192.jpg', 'https://forefdn.org/wp-content/uploads/2020/03/FORE_PR_Access_FINAL.pdf', 'https://forefdn.org/wp-content/uploads/2021/03/mother-3208577__340_alt.png', 'https://forefdn.org/wp-content/uploads/2019/08/ico-education.png', 'https://forefdn.org/wp-content/themes/fore/images/arrow_icon.png', 'https://forefdn.org/wp-content/uploads/2021/01/PolicyBrief1.png', 'https://forefdn.org/wp-content/uploads/2019/08/FORE_RFP_Access_to_Treatment-1-2.zip', 'https://forefdn.org/wp-content/uploads/2021/08/FORE-Audited-Financial-Statements-3.31.20.pdf', 'https://forefdn.org/news-updates//?res=resources', 'https://forefdn.org/wp-content/uploads/2020/07/Grantees2_featured.png', 'https://fore-portal.givingdata.com/campaign/family-community-prevention', 'https://forefdn.org/policy-brief-the-provision-of-medications-for-opioid-use-disorder-in-correctional-settings-in-the-time-of-covid-19-opportunities-and-solutions/', 

In [18]:
nx.write_gexf(G, 'forefdn_internal.gexf')